In [3]:
import cobra

# Quicker way to read in models
import pickle
def read_model(fileName, obj='none'):
    
    fileType = fileName.split('.')[-1]
    
    if fileType == 'sbml' or fileType == 'xml':
        model = cobra.io.read_sbml_model(fileName)
    elif fileType == 'json':
        model = cobra.io.load_json_model(fileName)
    elif fileType == 'yaml':
        model = cobra.io.load_yaml_model(fileName)
    elif fileType == 'mat':
        model = cobra.io.load_matlab_model(fileName)
    elif fileType == 'pkl':
        model = pickle.load(open(fileName, 'rb'))
    else:
        raise TypeError('Unrecognized file extension')
    
    if obj != 'none': model.objective = obj
    for rxn in model.boundary: rxn.bounds = (-1000., 1000.)
        
    return model


def find_auxotrophies(model):
    auxotrophy = []
    threshold = model.slim_optimize() * 0.01
    for rxn in model.boundary:
        rxn.bounds = (0.,1000.)
        if model.slim_optimize() < threshold: auxotrophy.append(rxn.reactants[0].id)
        rxn.bounds = (-1000.,1000.)
    
    auxotrophy = set(auxotrophy)
    print('Total auxotrophies:', len(auxotrophy))
    
    return auxotrophy

    

iCdG692 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG692.json')
iCdR700 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.json')

iCdG692.reactions.EX_cpd00011_e.bounds = (0., 0.) # anaerobic
iCdR700.reactions.EX_cpd00011_e.bounds = (0., 0.) # anaerobic

In [4]:
iCdG692_auxotrophies = find_auxotrophies(iCdG692)

Total auxotrophies: 13


In [5]:
iCdR700_auxotrophies = find_auxotrophies(iCdR700)

Total auxotrophies: 12


In [619]:
iCdG692 = read_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG692.json')

### Testing for Possible Energy Generation Cycles

In [234]:
# Dissipation reactions

cpd00009_c = iCdG692.metabolites.cpd00009_c # P
cpd00002_c = iCdG692.metabolites.cpd00002_c # ATP 
cpd00008_c = iCdG692.metabolites.cpd00008_c # ADP 
cpd00052_c = iCdG692.metabolites.cpd00052_c # CTP 
cpd00096_c = iCdG692.metabolites.cpd00096_c # CDP 
cpd00038_c = iCdG692.metabolites.cpd00038_c # GTP 
cpd00031_c = iCdG692.metabolites.cpd00031_c # GDP 
cpd00062_c = iCdG692.metabolites.cpd00062_c # UTP 
cpd00014_c = iCdG692.metabolites.cpd00014_c # UDP 
cpd00068_c = iCdG692.metabolites.cpd00068_c # ITP 
cpd00090_c = iCdG692.metabolites.cpd00090_c # IDP 
cpd00067_c = iCdG692.metabolites.cpd00067_c # H 
cpd00004_c = iCdG692.metabolites.cpd00004_c # NADH 
cpd00003_c = iCdG692.metabolites.cpd00003_c # NAD 
cpd00005_c = iCdG692.metabolites.cpd00005_c # NADPH
cpd00006_c = iCdG692.metabolites.cpd00006_c # NADP 
cpd00982_c = iCdG692.metabolites.cpd00982_c # FADH2
cpd00015_c = iCdG692.metabolites.cpd00015_c # FAD 

test_1 = cobra.Reaction('test_1')
test_1.bounds = (0., 1000.)
test_1.add_metabolites({ cpd00002_c: -1.0, cpd00008_c: 1.0, cpd00009_c: 1.0})

test_2 = cobra.Reaction('test_2')
test_2.bounds = (0., 1000.)
test_2.add_metabolites({cpd00052_c: -1.0, cpd00096_c: 1.0, cpd00009_c: 1.0})

test_3 = cobra.Reaction('test_3')
test_3.bounds = (0., 1000.)
test_3.add_metabolites({cpd00038_c: -1.0, cpd00031_c: 1.0, cpd00009_c: 1.0})

test_4 = cobra.Reaction('test_4')
test_4.bounds = (0., 1000.)
test_4.add_metabolites({cpd00062_c: -1.0, cpd00014_c: 1.0, cpd00009_c: 1.0})

test_5 = cobra.Reaction('test_5')
test_5.bounds = (0., 1000.)
test_5.add_metabolites({cpd00068_c: -1.0, cpd00090_c: 1.0, cpd00009_c: 1.0})

test_6 = cobra.Reaction('test_6')
test_6.bounds = (0., 1000.)
test_6.add_metabolites({cpd00004_c: -1.0, cpd00003_c: 1.0, cpd00067_c: 1.0})

test_7 = cobra.Reaction('test_7')
test_7.bounds = (0., 1000.)
test_7.add_metabolites({cpd00005_c: -1.0, cpd00006_c: 1.0, cpd00067_c: 1.0})

test_8 = cobra.Reaction('test_8')
test_8.bounds = (0., 1000.)
test_8.add_metabolites({cpd00982_c: -1.0, cpd00015_c: 1.0, cpd00067_c: 2.0})


In [235]:
from copy import deepcopy

model = deepcopy(iCdG692)
model.add_reactions([test_1,test_2,test_3,test_4,test_5,test_6,test_7,test_8])
for x in model.boundary: x.lower_bound = 0.
test_rxns = ['test_1','test_2','test_3','test_4','test_5','test_6','test_7','test_8']
edc = set()
for test_rxn in test_rxns:
    
    model.objective = test_rxn
    solution = model.optimize()
    active_rxns = set([rxn.id for rxn in model.reactions if abs(solution.fluxes[rxn.id]) > 1e-6])
    if len(active_rxns) > 0: edc |= active_rxns

edc = edc.difference(set(test_rxns))
edc = edc.difference(set([x.id for x in model.boundary]))
print('Total reactions in possible EDCs:', len(edc))

# Check reversibility 
reversible = []
irreversible = []
for rxn in edc:
    if iCdG692.reactions.get_by_id(rxn).lower_bound == 0.:
        irreversible.append(rxn)
    elif iCdG692.reactions.get_by_id(rxn).upper_bound == 0.:
        irreversible.append(rxn)
    else:
        reversible.append(rxn)
    

Total reactions in possible EDCs: 57


In [236]:
print('Total possible reversibility errors:', len(reversible))
for x in reversible: print(x, iCdG692.reactions.get_by_id(x).name)

Total possible reversibility errors: 40
rxn00611_c sn-Glycerol-3-phosphate:NAD+ 2-oxidoreductase
rxn08025_c Arbutin 6-phosphate glucohydrolase
rxn00213_c UTP:alpha-D-glucose-1-phosphate uridylyltransferase
rxn01132_c Maltose:phosphate 1-beta-D-glucosyltransferase
rxn10030_c b-ketoacyl synthetase (n-C18:1)
rxn00910_c 5-methyltetrahydrofolate:NADP+ oxidoreductase
rxn00355_c UTP:alpha-D-galactose-1-phosphate uridylyltransferase
rxn00139_c AMP:diphosphate phospho-D-ribosyltransferase
rxn00704_c alpha-D-Glucose 1-phosphate 1,6-phosphomutase
rxn00612_c sn-Glycerol-3-phosphate:NADP+ 2-oxidoreductase
rxn09348_c UDPgalactose pyrophosphohydrolase
rxn00499_c (S)-Lactate:NAD+ oxidoreductase
rxn05227_c Aspartate oxidase
rxn04043_c ADP:D-fructose-6-phosphate 1-phosphotransferase
rxn00605_c UDP-glucose:D-glucose-6-phosphate 1-alpha-D-glucosyltransferase
rxn00907_c 5,10-methylenetetrahydrofolate:NADP+ oxidoreductase
rxn09655_c Trehalose transport via proton symport
rxn01547_c Guanosine aminohydrolase


In [620]:
iCdG692.reactions.rxn00799_c.lower_bound = 0.
iCdG692.reactions.rxn05655_c.lower_bound = 0.
iCdG692.reactions.rxn01870_c.lower_bound = 0.
iCdG692.reactions.rxn00786_c.lower_bound = 0.
iCdG692.reactions.rxn00541_c.lower_bound = 0.
iCdG692.reactions.rxn13150_c.lower_bound = 0.
iCdG692.reactions.rxn02005_c.lower_bound = 0.
iCdG692.reactions.rxn10036_c.lower_bound = 0.
iCdG692.reactions.rxn00606_c.lower_bound = 0.
iCdG692.reactions.rxn00542_c.lower_bound = 0.
iCdG692.reactions.rxn00519_c.lower_bound = 0.
iCdG692.reactions.rxn01211_c.lower_bound = 0.
iCdG692.reactions.rxn00182_c.lower_bound = 0.
iCdG692.reactions.rxn08025_c.lower_bound = 0.
iCdG692.reactions.rxn00213_c.lower_bound = 0.
iCdG692.reactions.rxn10030_c.lower_bound = 0.
iCdG692.reactions.rxn00355_c.lower_bound = 0.
iCdG692.reactions.rxn05291_c.lower_bound = 0.
iCdG692.reactions.rxn00707_c.lower_bound = 0.
iCdG692.reactions.rxn08809_c.lower_bound = 0.
iCdG692.reactions.rxn09429_c.lower_bound = 0.
iCdG692.reactions.rxn00367_c.lower_bound = 0.
iCdG692.reactions.rxn00292_c.lower_bound = 0.

iCdG692.reactions.rxn01953_c.upper_bound = 1000.
iCdG692.reactions.rxn00292_c.upper_bound = 1000.

iCdG692.reactions.rxn05209_c.name = 'Sodium hydrogen symporter'
iCdG692.reactions.rxn03366_c.name = 'Dihydroxypicolinate oxidoreductase'
iCdG692.reactions.rxn03365_c.name = 'Dihydroxypicolinate oxidoreductase'

In [621]:
iCdG692.slim_optimize()

143.6921113236156

In [622]:
# Remove incorrect aerobic reactions 
iCdG692.remove_reactions(['rxn02304_c', 'rxn09180_c'])

# Add other detoxification reaction
cpd00001_c = iCdG692.metabolites.cpd00001_c # H2O
cpd00023_c = iCdG692.metabolites.cpd00023_c # L-Glutamate
cpd00003_c = iCdG692.metabolites.cpd00003_c # NAD
cpd00004_c = iCdG692.metabolites.cpd00004_c # NADH
cpd00024_c = iCdG692.metabolites.cpd00024_c # 2-Oxoglutarate
cpd00067_c = iCdG692.metabolites.cpd00067_c # H+
cpd00013_c = iCdG692.metabolites.cpd00013_c # Ammonia

iCdG692.remove_reactions(['rxn30075_c'])
rxn30075_c = cobra.Reaction('rxn30075_c')
rxn30075_c.name = 'Glutamate dehydrogenase (NAD)'
rxn30075_c.gene_reaction_rule = '645463.3.peg.194'
rxn30075_c.lower_bound = 0.
rxn30075_c.upper_bound = 1000.
rxn30075_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd00003_c: -1.0,
    cpd00023_c: -1.0,
    cpd00024_c: 1.0,
    cpd00004_c: 1.0,
    cpd00067_c: 1.0,
    cpd00013_c: 1.0
})
iCdG692.add_reactions([rxn30075_c])

iCdG692.reactions.rxn30075_c.annotation['sbo'] = 'SBO:0000176' 
iCdG692.reactions.rxn30075_c.annotation['seed.reaction'] = 'rxn30075'
iCdG692.reactions.rxn30075_c.annotation['bigg.reaction'] = 'GLUDx'


In [623]:
# Correct proline auxotrophy
iCdG692.remove_reactions(['rxn00471_c'])

In [624]:
iCdG692.slim_optimize()

143.69211132361568

In [570]:
iCdG692

Name,iCdG692
Memory address,0x07f79a51b1310
Number of metabolites,1235
Number of reactions,1305
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular, periplasm"


In [625]:
mdm = ['cpd00001_e', # water
       'cpd00065_e', # L-Tryptophan
       'cpd00060_e', # L-Methionine
       'cpd00322_e', # L-Isoleucine
       'cpd00129_e', # L-Proline
       'cpd00156_e', # L-Valine
       'cpd00107_e', # L-Leucine
       'cpd00084_e', # L-Cysteine 
       'cpd00149_e', # Cobalt
       'cpd00099_e', # Chloride
       'cpd10515_e', # Iron
       'cpd00030_e', # Manganese
       'cpd00254_e', # Magnesium
       'cpd00063_e', # Calcium
       'cpd00205_e', # Potassium
       'cpd00009_e', # Phosphate
       'cpd00971_e', # Sodium
       'cpd00242_e', # Carbonate
       'cpd00104_e', # Biotin
       'cpd00644_e', # Pantothenate
       'cpd00263_e', # Pyridoxine
       'cpd00027_e'] # D-Glucose (Carbohydrate C-source)
mdm = ['EX_' + x for x in mdm]
for rxn in iCdG692.boundary:
    rxn.bounds = (-1000., 1000.)
    if not rxn.id in mdm:
        rxn.bounds = (0., 1000.)

In [626]:
iCdG692.remove_reactions(['rxn00916_c'])

cpd00002_c = iCdG692.metabolites.cpd00002_c
cpd00013_c = iCdG692.metabolites.cpd00013_c
cpd00497_c = iCdG692.metabolites.cpd00497_c
cpd00012_c = iCdG692.metabolites.cpd00012_c
cpd00018_c = iCdG692.metabolites.cpd00018_c
cpd00067_c = iCdG692.metabolites.cpd00067_c
cpd00126_c = iCdG692.metabolites.cpd00126_c

rxn00916_c = cobra.Reaction('rxn00916_c')
rxn00916_c.name = 'Xanthosine-phosphate:ammonia ligase (AMP-forming)'
rxn00916_c.lower_bound = 0.
rxn00916_c.upper_bound = 1000.
rxn00916_c.add_metabolites({
    cpd00002_c: -1.0,
    cpd00013_c: -1.0,
    cpd00497_c: -1.0,
    cpd00012_c: 1.0,
    cpd00018_c: 1.0,
    cpd00067_c: 3.0,
    cpd00126_c: 1.0
})
iCdG692.add_reactions([rxn00916_c])
iCdG692.reactions.rxn00916_c.annotation['sbo'] = 'SBO:0000176' 
iCdG692.reactions.rxn00916_c.annotation['seed.reaction'] = 'rxn00916'
iCdG692.reactions.rxn00916_c.annotation['kegg.reaction'] = 'R01230'


In [599]:
gapfilled_rxns = pfba_gapfill(iCdG692, universal_GramPos) # Biomass alone

New reactions: 3
New objective value: 6.1000000000000005


In [600]:
gapfilled_rxns

{'rxn00116_c', 'rxn01347_c', 'rxn01952_c'}

In [627]:
rxn00116_c = deepcopy(universal_GramPos.reactions.rxn00116_c)
rxn01347_c = deepcopy(universal_GramPos.reactions.rxn01347_c)
rxn01952_c = deepcopy(universal_GramPos.reactions.rxn01952_c)
iCdG692.add_reactions([rxn00116_c, rxn01347_c, rxn01952_c])

iCdG692.reactions.rxn00116_c.annotation['sbo'] = 'SBO:0000176' 
iCdG692.reactions.rxn00116_c.annotation['seed.reaction'] = 'rxn00116'
iCdG692.reactions.rxn00116_c.annotation['kegg.reaction'] = 'R00155'
iCdG692.reactions.rxn00116_c.annotation['bigg.reaction'] = 'NDP7'

iCdG692.reactions.rxn01347_c.annotation['sbo'] = 'SBO:0000176' 
iCdG692.reactions.rxn01347_c.annotation['seed.reaction'] = 'rxn01347'

iCdG692.reactions.rxn01952_c.annotation['sbo'] = 'SBO:0000176' 
iCdG692.reactions.rxn01952_c.annotation['seed.reaction'] = 'rxn01952'
iCdG692.reactions.rxn01952_c.annotation['kegg.reaction'] = 'R02706'

In [640]:
iCdG692.slim_optimize()

38.307060909495995

In [643]:
iCdG692.id = 'iCdG' + str(len(iCdG692.genes))

In [644]:
iCdG692

Name,iCdG693
Memory address,0x07f79a12d2610
Number of metabolites,1235
Number of reactions,1309
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular, periplasm"


In [645]:
cobra.io.write_sbml_model(iCdG692, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG693.sbml')
cobra.io.save_json_model(iCdG692, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG693.json')